中国证监会《上市公司行业分类指引》(2012年修订)将上市公司划分为19个门类和90个大类。中基协基金估值行业分类指数依据《上市公司行业分类指引》中的门类划分，编制16个门类指数(不包括制造业);依据制造业门类下的大类划分，编制27个大类指数，共有43条行业分类指数。

In [1]:
## load required libraries
import pandas as pd
import warnings
warnings.simplefilter("ignore")

In [2]:
## load amac_info.csv
industry_name_map = {}
data = pd.read_csv('entire/amac_info.csv')
for index, row in data.iterrows():
	industry_name_map[row['industry']] = [row['description'], index]
industry_n = len(industry_name_map.keys())
print(industry_name_map)

{'H11030': ['AMAC农林牧渔指数', 0], 'H11031': ['AMAC采矿指数', 1], 'H11041': ['AMAC水电煤气指数', 2], 'H11042': ['AMAC建筑指数', 3], 'H11043': ['AMAC交运仓储指数', 4], 'H11044': ['AMAC信息技术指数', 5], 'H11045': ['AMAC批发零售贸易指数', 6], 'H11046': ['AMAC金融保险指数', 7], 'H11047': ['AMAC房地产指数', 8], 'H11049': ['AMAC文化体育指数', 9], 'H11050': ['AMAC综合企业指数', 10], 'H30036': ['AMAC餐饮住宿指数', 11], 'H30037': ['AMAC租赁商务指数', 12], 'H30038': ['AMAC科研技术指数', 13], 'H30039': ['AMAC水利环境指数', 14], 'H30040': ['AMAC居民服务指数', 15], 'H30041': ['AMAC食品加工指数', 16], 'H30042': ['AMAC食品制造指数', 17], 'H30043': ['AMAC饮料制造指数', 18], 'H30044': ['AMAC纺织指数', 19], 'H30045': ['AMAC纺织服装指数', 20], 'H30046': ['AMAC皮革指数', 21], 'H30047': ['AMAC木材指数', 22], 'H30048': ['AMAC家具指数', 23], 'H30049': ['AMAC造纸指数', 24], 'H30050': ['AMAC印刷指数', 25], 'H30051': ['AMAC文教用品指数', 26], 'H30052': ['AMAC石化指数', 27], 'H30053': ['AMAC化学制品指数', 28], 'H30054': ['AMAC医药制造指数', 29], 'H30055': ['AMAC化纤指数', 30], 'H30056': ['AMAC橡胶塑料指数', 31], 'H30057': ['AMAC非金属指数', 32], 'H30058': ['AMAC黑色金属指数', 33], 'H30059':

In [3]:
## load amac_info.csv
data = pd.read_csv('entire/amac.csv')
data = data[(data['day'] >= '2018-01-01') & (data['day'] <= '2019-12-31')]
data['ratio'] = (data['end'] - data['start'])/data['start']
data.head()

,industry,day,start,end,ratio
49,H11030,2019-12-31,2834.42,2877.95,0.015358
50,H11030,2019-12-30,2775.79,2815.00,0.014126
51,H11030,2019-12-27,2724.33,2779.00,0.020067
52,H11030,2019-12-26,2733.94,2715.82,-0.006628
53,H11030,2019-12-25,2746.24,2727.54,-0.006809


### Preprocess Data

In [4]:
data_asso = {}
for index, row in data.iterrows():
    if row['day'] not in data_asso:
        data_asso[row['day']] = [False] * industry_n * 2
    if row['ratio'] >= 0:
        data_asso[row['day']][2*industry_name_map[row['industry']][1]] = True
    else:
        data_asso[row['day']][2*industry_name_map[row['industry']][1]+1] = True

cols = []
for key in industry_name_map.keys():
    cols.append(industry_name_map[key][0] + '-涨')
    cols.append(industry_name_map[key][0] + '-跌')

data_asso_df = pd.DataFrame(columns = cols)
line_n = 0
for val in data_asso.values():
    data_asso_df.loc[line_n] = val
    line_n += 1

data_asso_df.head()

,AMAC农林牧渔指数-涨,AMAC农林牧渔指数-跌,AMAC采矿指数-涨,AMAC采矿指数-跌,AMAC水电煤气指数-涨,AMAC水电煤气指数-跌,AMAC建筑指数-涨,AMAC建筑指数-跌,AMAC交运仓储指数-涨,AMAC交运仓储指数-跌,...,AMAC汽车制造指数-涨,AMAC汽车制造指数-跌,AMAC运输设备指数-涨,AMAC运输设备指数-跌,AMAC电气机械指数-涨,AMAC电气机械指数-跌,AMAC电子设备指数-涨,AMAC电子设备指数-跌,AMAC仪器仪表指数-涨,AMAC仪器仪表指数-跌
0,True,False,True,False,True,False,True,False,True,False,...,True,False,True,False,False,True,True,False,True,False
1,True,False,True,False,True,False,True,False,True,False,...,True,False,True,False,True,False,True,False,True,False
2,True,False,True,False,False,True,False,True,True,False,...,False,True,False,True,False,True,False,True,False,True
3,False,True,True,False,True,False,True,False,True,False,...,True,False,True,False,True,False,True,False,True,False
4,False,True,False,True,False,True,False,True,False,True,...,True,False,False,True,True,False,True,False,True,False


### Generate Frequency Items
- By Apriori

In [5]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets = apriori(data_asso_df, min_support = 0.4, use_colnames = True)
frequent_itemsets.head()

,support,itemsets
0,0.546201,(AMAC农林牧渔指数-涨)
1,0.453799,(AMAC农林牧渔指数-跌)
2,0.542094,(AMAC采矿指数-涨)
3,0.457906,(AMAC采矿指数-跌)
4,0.527721,(AMAC水电煤气指数-涨)


- By FP-Growth

In [6]:
from mlxtend.frequent_patterns import fpgrowth
frequent_itemsets = fpgrowth(data_asso_df, min_support = 0.4, use_colnames = True)
frequent_itemsets.head()

,support,itemsets
0,0.583162,(AMAC皮革指数-涨)
1,0.554415,(AMAC纺织指数-涨)
2,0.550308,(AMAC专用设备指数-涨)
3,0.550308,(AMAC文教用品指数-涨)
4,0.546201,(AMAC农林牧渔指数-涨)


### Generate Association Rules

In [7]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_itemsets, metric = 'confidence', min_threshold = 0.8)
rules.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(AMAC皮革指数-涨),(AMAC纺织指数-涨),0.583162,0.554415,0.472279,0.809859,1.460746,0.148966,2.343448
1,(AMAC纺织指数-涨),(AMAC皮革指数-涨),0.554415,0.583162,0.472279,0.851852,1.460746,0.148966,2.813655
2,(AMAC专用设备指数-涨),(AMAC纺织指数-涨),0.550308,0.554415,0.468172,0.850746,1.534494,0.163074,2.985421
3,(AMAC纺织指数-涨),(AMAC专用设备指数-涨),0.554415,0.550308,0.468172,0.844444,1.534494,0.163074,2.890877
4,(AMAC专用设备指数-涨),(AMAC皮革指数-涨),0.550308,0.583162,0.464066,0.843284,1.446053,0.143147,2.659822


In [8]:
rules['antecedents_len'] = rules['antecedents'].apply(lambda x: len(x))
rules['consequents_len'] = rules['consequents'].apply(lambda x: len(x))
rules_s = rules[(rules['antecedents_len'] <= 1) & (rules['consequents_len'] == 1) & (rules['support'] >= 0.44) & (rules['confidence'] >= 0.84) & (rules['lift'] >= 1.5)]
rules_s
rules_s.sort_values('lift', ascending = False).iloc[:10,[0,1,4,5,6]]

,antecedents,consequents,support,confidence,lift
791,(AMAC通用设备指数-涨),(AMAC仪器仪表指数-涨),0.449692,0.848837,1.687280
790,(AMAC仪器仪表指数-涨),(AMAC通用设备指数-涨),0.449692,0.893878,1.687280
313,(AMAC医药制造指数-涨),(AMAC批发零售贸易指数-涨),0.464066,0.875969,1.647092
314,(AMAC批发零售贸易指数-涨),(AMAC医药制造指数-涨),0.464066,0.872587,1.647092
347,(AMAC通用设备指数-涨),(AMAC化学制品指数-涨),0.462012,0.872093,1.646160
346,(AMAC化学制品指数-涨),(AMAC通用设备指数-涨),0.462012,0.872093,1.646160
344,(AMAC专用设备指数-涨),(AMAC通用设备指数-涨),0.474333,0.861940,1.626996
345,(AMAC通用设备指数-涨),(AMAC专用设备指数-涨),0.474333,0.895349,1.626996
792,(AMAC仪器仪表指数-涨),(AMAC专用设备指数-涨),0.449692,0.893878,1.624322
98,(AMAC专用设备指数-涨),(AMAC批发零售贸易指数-涨),0.472279,0.858209,1.613698
